In [1]:
!pip install transformers -q
!pip install lime -q

You should consider upgrading via the '/opt/conda/envs/rapids/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/envs/rapids/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
%cd ..
from src.model_wrapper import Model
from src.explainer_lime import Lime_Explanation

/usr/src/app/data/XAI_Thesis


In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import os
import logging

In [4]:
# config
chunk_tag = 'A' # the possible chunks are A, B, C, D, E with 2700 rows each
target_path_for_combined_data = f'./data/GermanQuAD_raw_combined/GermanQuAD_raw_combined_{chunk_tag}.json'
model_name = 'deepset/gelectra-base-germanquad'

number_of_features = 10 # the default value
number_of_samples = 5000 # the default value

In [5]:
# get raw data combined in one dataframe (test + train)
GQuAD_chunk_data = pd.read_json(target_path_for_combined_data)
GQuAD_chunk_data.head(5)

,question_id,question,answers,context,document_id,is_impossible,usage
0,67118,Warum sind höhere Temperaturen für die Bildung...,"[{'answer_id': 72940, 'document_id': 73863, 'q...",Gletscher\n\n=== Akkumulation und Metamorphose...,73863,False,train
1,41385,Mit welchen Päpsten hat sich Elisabeth II getr...,"[{'answer_id': 43009, 'document_id': 40918, 'q...",Elisabeth_II.\n\n== Persönlichkeit und öffentl...,40918,False,test
2,40503,Wie lange dauerte der Bau des Camp de Les Corts,"[{'answer_id': 40046, 'document_id': 40992, 'q...",FC_Barcelona\n\n=== Frühere Spielstätten ===\n...,40992,False,test
3,36706,In welchem Bundesstaat liegt Melbourne?,"[{'answer_id': 37054, 'document_id': 40673, 'q...",Melbourne\n\n=== Stadtregierung ===\nDie Metro...,40673,False,test
4,67578,Wann begann Hitler Truppen Richtung Sowjetunio...,"[{'answer_id': 73435, 'document_id': 57930, 'q...",Deutsch-sowjetischer_Nichtangriffspakt\n\n=== ...,57930,False,train


In [6]:
# get important instances
GELECTRA_MODEL = Model(model_name)

# get lime explainer
LIME_EXPLAINER = Lime_Explanation(model = GELECTRA_MODEL,
                                  class_names = ["Start-Token","End-Token"], 
                                  num_samples = number_of_samples, # hier noch eine begründung warum
                                  num_features = number_of_features, # default value
                                  random_state = 0)

In [7]:
# helper functions to enhance_data_frame

def get_prediction(row):
    model_output = GELECTRA_MODEL.predict(row.question, row.context)
    
    answer = GELECTRA_MODEL.get_predicted_tokens(model_output[0],model_output[1])
    answer_string = GELECTRA_MODEL.get_answer_string(model_output[0],model_output[1])
    
    return answer_string

def get_lime_explanation(row):
    
    try:
        explanation = LIME_EXPLAINER.explain(row.question, row.context)

        explanation_start_token = explanation.as_list(label = 0)
        explanation_end_token = explanation.as_list(label = 1)

        return pd.Series([explanation_start_token, explanation_end_token])
    except:
        print(row.question_id)
        return pd.Series([None, None])
    
def enhance_data_frame(GQuAD_data):
    GQuAD_data['prediction'] = GQuAD_data.apply(lambda row: get_prediction(row), axis=1)
    GQuAD_data[['start_token_exlanation_lime', 'end_token_explanation_lime']] = GQuAD_data.apply(lambda row: get_lime_explanation(row), axis=1)
    
    return GQuAD_data

In [8]:
# helper function: parallizes the process (DOES NOT BRING A BENEFIT)
# def parallize_dataframe(dataframe, func, n_cores):
#     df_split = np.array_split(dataframe, n_cores)
#     pool = multiprocessing.Pool(n_cores)
    
#     dataframe = pd.concat(pool.map(func, df_split))
#     pool.close()
#     pool.join()
    
#     return dataframe

In [9]:
# creat new directory with specifications
dir_name = f'data/German_QuAD_with_lime/lime_ns_{number_of_samples}_nf_{number_of_features}'
if not os.path.isdir(dir_name):
    os.mkdir(dir_name)

In [10]:
# create a  logger
logging.basicConfig(filename=f'{dir_name}/{chunk_tag}_0_log.log', # file name
                    filemode='a', # file mode a == append instead of overwrite
                    level=logging.DEBUG,
                    datefmt='%Y-%m-%d %H:%M:%S'
                   )

In [14]:
# split df in chunks for better handeling (and backing up after each chunk)
list_of_dfs = np.array_split(GQuAD_chunk_data, 45)
len(list_of_dfs)

# TODO -> wenn ich weiß wie lange eine row auf dem cluster braucht, kann ich abschätzen wie groß die sub_chunks sein sollen

45

In [15]:
# iterate throug the dfs
for i in range(len(list_of_dfs)):
    try:
        current_df_chunck = list_of_dfs[i]

        first_index = current_df_chunck.index[0]
        last_index = current_df_chunck.index[-1]

        logging.info(f'Starting on new sub_chunk. The first index is: {first_index}')
        
        # enhance df sub-chunk
        df_sub_chuck_explained = enhance_data_frame(list_of_dfs[i])

        # safe them to a file with the correct name (as a counter)
        df_sub_chuck_explained.to_json(f'{dir_name}/{chunk_tag}_{i + 1}_von_{len(list_of_dfs)}_GermanQuAD_explained_lime.json')

        # log the index of the last row of the df
        logging.info(f'One sub_chunk has been finished and safed. The last index is: {last_index}')
                     
    except Exception as e:
        logging.critical(e, exc_info=True)
        

40503
47289
40480
47414
56619
52417
52874
58985
54600
53516
